In [ ]:
import sys
sys.path.append('/home1/yxiao977/labwork')
import test_func 

In [ ]:
request = {
    'hic_path':'/home1/yxiao977/sc1/train_akita/data/5000res_0.5thres_hic_filter_both_bin.cool',
    'cis_total_path':'/home1/yxiao977/sc1/train_akita/data/5000res_bins_cistotal_ratio.bed'
}

test_func.test_cis_total_filter(request)

In [1]:
### Test need to delete later
import cooler
import numpy as np
clr = cooler.Cooler('/home1/yxiao977/sc1/train_akita/data/GSE152150_HiC-Dplus.smic1.1N.mapq_30.1000.mcool::/resolutions/5000')

In [6]:
t = None
t == None

True

In [2]:
import numpy as np
import pandas as pd
import cooltools
import cooler
import logging
import functools
def _bins_cis_total_ratio_filter(clr, thres):
    """
    Filter out bins with low cis-to-total coverage ratio from a Cooler object.

    Parameters
    ----------
    clr : cooler.Cooler
        A Cooler object containing Hi-C contact matrices.
    thres : float
        The threshold cis-to-total coverage ratio below which bins are considered bad.

    Returns
    -------
    numpy.ndarray
        An array of bin mask.
    """
    coverage = cooltools.coverage(clr)
    cis_total_cov = coverage[0] / coverage[1]
    bins_mask = cis_total_cov > thres

    return bins_mask

/home1/yxiao977/.conda/envs/akita/lib/python3.9/site-packages/cooltools/lib/numutils.py:652: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iterative_correction_symmetric(
/home1/yxiao977/.conda/envs/akita/lib/python3.9/site-packages/cooltools/lib/numutils.py:727: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iterative_correction_asymmetric(x, max_iter=10

In [3]:
def _pixels_filter(chunk_pixels, bins_table_size, bin_mask):
    """
    Filter out pixels that belong to bad bins.

    Parameters:
    -----------
    bin_table : numpy.ndarray
        A 2D array of shape (n_bins, n_features) containing the features of each bin.
    chunk_pixels : pandas.DataFrame
        A DataFrame containing the pixels to filter.
    bin_mask : numpy.ndarray
        A boolean array of shape (n_bins,) indicating which bins are bad.

    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing only the pixels that belong to good bins.
    """
    bad_bins_index = np.array(range(bins_table_size))[bin_mask]
    pixels_mask = chunk_pixels['bin1_id'].isin(bad_bins_index) * chunk_pixels['bin2_id'].isin(bad_bins_index)
    return chunk_pixels[pixels_mask]

def pixels_filter_generator(bins_table_size, bin_mask):
    """
    Returns a partial function that filters pixels based on the provided bins table and bin mask.

    Parameters:
    -----------
    bins_table_size : int
        A integer value indicates the size of bin table.
    bin_mask : numpy.ndarray
        A numpy array containing the bin mask.

    Returns:
    --------
    partial function
        A partial function that filters pixels based on the provided bins table and bin mask.
    """

    return functools.partial(_pixels_filter, bins_table_size=bins_table_size, bin_mask=bin_mask)

class PixelsIterator:
    def __init__(self, clr_pixels_selector, pixels_size, chunksize):
        self.chunksize = chunksize
        self.max = pixels_size
        self.pixels = clr_pixels_selector

    def __iter__(self):
        self.pivot = 0
        return self

    def __next__(self):
        if (self.pivot + self.chunksize) < self.max:
            pivot = self.pivot
            self.pivot += self.chunksize
            return self.pixels[pivot : self.pivot]
        elif self.pivot < self.max:
            pivot = self.pivot
            self.pivot = self.max
            return self.pixels[pivot : self.pivot]
        else:
            raise StopIteration

In [1]:
import logging
logger = logging.getLogger('one')
logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s : %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)


In [7]:
~np.array([True, False])

array([False,  True])

In [8]:
import numpy as np
bin_mask = _bins_cis_total_ratio_filter(clr, 0.5)
bad_bins_index = np.array(range(clr.bins().shape[0]))[~bin_mask]

/tmp/SLURM_16817575/ipykernel_40495/1550455361.py:24: RuntimeWarning: invalid value encountered in true_divide
  cis_total_cov = coverage[0] / coverage[1]


In [11]:
bad_bins_i

array([    12,     13,     23, ..., 147181, 147182, 147183])

In [7]:
bin_mask = _bins_cis_total_ratio_filter(clr, 0.5)
bins_table = clr.bins()[:]
tot_pixels = clr.pixels().shape[0]

/tmp/SLURM_16768932/ipykernel_4290/1550455361.py:24: RuntimeWarning: invalid value encountered in true_divide
  cis_total_cov = coverage[0] / coverage[1]


In [34]:
pixels_chunks = PixelsIterator(clr.pixels(), tot_pixels, chunksize=10_000_000)
pixels_filter = pixels_filter_generator(bins_table.shape[0], bin_mask)


In [4]:
import cooler
t_clr = cooler.Cooler('/home1/yxiao977/sc1/test_func/test_create.cool')

In [6]:
len(bin_mask) - np.sum(bin_mask)

147184

In [26]:
t_clr.pixels().shape

(449402872,)

In [24]:
t_clr.bins()[13253]

,chrom,start,end,weight
13253,chr5_pilon,4260000,4265000,NaN


In [25]:
t_clr.pixels().fetch('chr5_pilon:4260000-4265000')

,bin1_id,bin2_id,count


In [31]:
np.array([True, False]) * np.array([True, True])

array([ True, False])

In [22]:
clr.pixels().fetch('chr5_pilon:4260000-4265000')

,bin1_id,bin2_id,count
71972446,13253,15233,1
71972447,13253,43947,1
71972448,13253,96858,1
71972449,13253,118939,1


In [12]:
import pandas as pd
cis_total_cov = pd.read_table('/home1/yxiao977/sc1/train_akita/data/5000res_bins_cistotal_ratio.bed', sep='\t')

In [16]:
cis_total_cov[cis_total_cov['cis-total-ratio'].isna()]

,chrom,start,end,cis-total-ratio
578,chr1_pilon,2890000,2895000,NaN
579,chr1_pilon,2895000,2900000,NaN
580,chr1_pilon,2900000,2905000,NaN
581,chr1_pilon,2905000,2910000,NaN
582,chr1_pilon,2910000,2915000,NaN
...,...,...,...,...
146920,chr91_pilon,80000,85000,NaN
146921,chr91_pilon,85000,90000,NaN
147146,chr92_pilon,505000,510000,NaN
147176,chr93_pilon,130000,135000,NaN


In [20]:
sums = []
for i, row in cis_total_cov[cis_total_cov['cis-total-ratio'].isna()].iterrows():
    sums.append(np.nansum(clr.pixels().fetch(row[:3])['count']))

In [27]:
cis_total_cov[cis_total_cov['cis-total-ratio'].isna()].index[np.where(np.array(sums) > 0)]

Index([1248, 70460, 110322], dtype='int64')

In [30]:
cis_total_cov.loc[1248]

chrom              chr1_pilon
start                 6240000
end                   6245000
cis-total-ratio           NaN
Name: 1248, dtype: object

In [31]:
clr.pixels().fetch('chr1_pilon:6240000-6245000')

,bin1_id,bin2_id,count
8733327,1248,1248,1


In [15]:
len(cis_total_cov[cis_total_cov['cis-total-ratio'].isna()]) + len(cis_total_cov[cis_total_cov['cis-total-ratio'] <= 0.5])

10533

In [12]:
len(cis_total_cov)

147184

In [16]:
len(bin_mask) - np.sum(bin_mask)

10533

In [10]:
from multiprocessing import Pool

def f(x):
    return x

with Pool(4) as p:
    print(p.map(f, [1,2,3,4,5,6,7,8,9,10,11,12]))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [ ]:
pixels_mask = [bin_mask[i] for i in chunk_pixels['bin1_id'].values]

In [9]:
assert False

AssertionError: 

In [16]:
import numpy as np
assert np.isnan([np.nan, np.nan]).all()

In [1]:
import sys
sys.path.append('/home1/yxiao977/labwork/Akita_dinoflagellate/')
import test_func

test_func.test_cis_total_filter(0.5)

/home1/yxiao977/.conda/envs/akita/lib/python3.9/site-packages/cooltools/lib/numutils.py:652: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iterative_correction_symmetric(
/home1/yxiao977/.conda/envs/akita/lib/python3.9/site-packages/cooltools/lib/numutils.py:727: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iterative_correction_asymmetric(x, max_iter=10

AssertionError: 

In [5]:
import numpy as np
np.array([1,1,1,1]).all() == np.array([1,1,1,1]).all()

True

In [2]:
import cooler 

clr_file = "/home1/yxiao977/sc1/train_akita/test_data/test_data_util.cool"
clr_test = cooler.Cooler(clr_file)

In [3]:
clr_test.bins()[:]

,chrom,start,end
0,chr1,0,5
1,chr1,5,8
2,chrX,0,5
3,chrX,5,10


In [ ]:
clr._load_attrs(clr.root.rstrip("/") + "/bins/weight")["ignore_diags"]